In [1]:
import numpy as np
from scipy.sparse.linalg import *
from scipy.constants import *
from matplotlib import pyplot as plt
from scipy.spatial import Delaunay
from scipy.interpolate import griddata
%matplotlib inline

In [2]:
from QCircuit import *
from AdaptiveParametricSpaceMapper import AdaptiveParametricSpaceMapper

In [ ]:
FQ3JJ = QCircuit()
FQ3JJ.add_element(QJosephsonJunction('JJ1'), ['GND', '1'])
FQ3JJ.add_element(QJosephsonJunction('JJ2'), ['1', '2'])
FQ3JJ.add_element(QJosephsonJunction('JJ3'), ['2', '3'])
FQ3JJ.add_element(QCapacitance('C1'), ['GND', '1'])
FQ3JJ.add_element(QCapacitance('C2'), ['1', '2'])
FQ3JJ.add_element(QCapacitance('C3'), ['2', '3'])
        
phi1 = QVariable('φ1')
phi2 = QVariable('φ2')
phix = QVariable('φx')
phi1.create_grid(32, 1)
phi2.create_grid(32, 1)
FQ3JJ.add_variable(phi1)
FQ3JJ.add_variable(phi2)
FQ3JJ.add_variable(phix)
FQ3JJ.map_nodes_linear(['GND', '1', '2', '3'], 
                       ['φ1', 'φ2', 'φx'], 
                       np.asarray([[0,0,0],[1,0,0],[1,1,0],[0,0,1]]))

EjEc_ratio_steps = 16
alpha_steps = 16
flux_steps = 16
energies = np.zeros((EjEc_ratio_steps,alpha_steps,flux_steps,2), dtype=np.float)

EjEc_ratios, alphas, qubit_fluxes = np.meshgrid(np.logspace(1,3,EjEc_ratio_steps),
                                             np.linspace(0.5, 1, alpha_steps),
                                             np.linspace(0,np.pi,flux_steps))


for EjEc_ratio_id, EjEc_ratio in enumerate(np.logspace(1, 3, EjEc_ratio_steps)):
    Ej = EjEc_ratio*1e9
    Ec = 1e9
    FQ3JJ.find_element('JJ1').set_critical_current(Ej)
    FQ3JJ.find_element('JJ2').set_critical_current(Ej)
    FQ3JJ.find_element('C1').set_capacitance(2/Ec)
    FQ3JJ.find_element('C2').set_capacitance(2/Ec)
    for alpha_id, alpha in enumerate(np.linspace(0.5, 1, alpha_steps)):
        FQ3JJ.find_element('JJ3').set_critical_current(Ej*alpha)
        FQ3JJ.find_element('C3').set_capacitance(2*alpha/Ec)
        for qubit_flux_id, qubit_flux in enumerate(np.linspace(0, np.pi, flux_steps)):
            #print('EjEc_ratio id: {0: 2d}/{1: 2d}, alpha id: {2: 2d}/{3: 2d}, flux id: {4: 2d}/{5: 2d}'.format(
            #        EjEc_ratio_id, EjEc_ratio_steps, alpha_id, alpha_steps, qubit_flux_id, flux_steps))
            phix.set_parameter(qubit_flux, 0)
            FQ3JJ.calculate_potentials()
            [eigenenergies, eigenfunctions] = FQ3JJ.diagonalize_phase()
            energies[EjEc_ratio_id, alpha_id, qubit_flux_id, :] = eigenenergies

In [6]:
FQ3JJ = QCircuit()
FQ3JJ.add_element(QJosephsonJunction('JJ1'), ['GND', '1'])
FQ3JJ.add_element(QJosephsonJunction('JJ2'), ['1', '2'])
FQ3JJ.add_element(QJosephsonJunction('JJ3'), ['2', '3'])
FQ3JJ.add_element(QCapacitance('C1'), ['GND', '1'])
FQ3JJ.add_element(QCapacitance('C2'), ['1', '2'])
FQ3JJ.add_element(QCapacitance('C3'), ['2', '3'])
        
phi1 = QVariable('φ1')
phi2 = QVariable('φ2')
phix = QVariable('φx')
phi1.create_grid(32, 1)
phi2.create_grid(32, 1)
FQ3JJ.add_variable(phi1)
FQ3JJ.add_variable(phi2)
FQ3JJ.add_variable(phix)
FQ3JJ.map_nodes_linear(['GND', '1', '2', '3'], 
                       ['φ1', 'φ2', 'φx'], 
                       np.asarray([[0,0,0],[1,0,0],[1,1,0],[0,0,1]]))

def CalcSingleGap(parameters):
    Ej = 10**parameters[0]
    Ec = 2e9
    alpha = parameters[1]
    qubit_flux = parameters[2]
    FQ3JJ.find_element('JJ1').set_critical_current(Ej)
    FQ3JJ.find_element('JJ2').set_critical_current(Ej)
    FQ3JJ.find_element('C1').set_capacitance(2/Ec)
    FQ3JJ.find_element('C2').set_capacitance(2/Ec)
    FQ3JJ.find_element('JJ3').set_critical_current(Ej*alpha)
    FQ3JJ.find_element('C3').set_capacitance(2*alpha/Ec)   
    phix.set_parameter(qubit_flux, 0)
    FQ3JJ.calculate_potentials()
    [eigenenergies, eigenfunctions] = FQ3JJ.diagonalize_phase()
    print('Ej: {0:8.3g}, Ec: {1:8.3g}, alpha: {2:8.3g}, flux: {3:8.3g}. Gap: {4:8.3g}'.format(
            Ej, Ec, alpha, qubit_flux, np.abs(eigenenergies[1]-eigenenergies[0])))

GapMapper = AdaptiveParametricSpaceMapper([('Ej', 9, 12), 
                                           ('alpha', 0.5, 1.0), 
                                           ('qubit_flux', 0, 2*np.pi)], 
                                            CalcSingleGap, 
                                            lambda x: np.log10(np.abs(x[:,1]-x[:,0])))

In [7]:
GapMapper.run(max_vertices = 500)

Ej: 3.16e+10, Ec:    2e+09, alpha:     0.75, flux:     3.14. Gap: 1.55e+09
Ej:    1e+09, Ec:    2e+09, alpha:      0.5, flux:        0. Gap: 6.03e+08
Ej:    1e+12, Ec:    2e+09, alpha:      0.5, flux:        0. Gap: 3.26e+10
Ej:    1e+09, Ec:    2e+09, alpha:        1, flux:        0. Gap: 7.95e+08
Ej:    1e+12, Ec:    2e+09, alpha:        1, flux:        0. Gap: 3.33e+10
Ej:    1e+09, Ec:    2e+09, alpha:      0.5, flux:     6.28. Gap: 6.03e+08
Ej:    1e+12, Ec:    2e+09, alpha:      0.5, flux:     6.28. Gap: 3.26e+10
Ej:    1e+09, Ec:    2e+09, alpha:        1, flux:     6.28. Gap: 7.95e+08
Ej:    1e+12, Ec:    2e+09, alpha:        1, flux:     6.28. Gap: 3.33e+10
[[  2.80464679e+10 +7.15532278e-06j   3.61337672e+10 -2.43393083e-07j]
 [ -3.77509524e+10 -1.00207528e-05j  -6.14444849e+10 -1.28177938e-06j]
 [  2.80454103e+10 +1.59056293e-06j   3.63242920e+10 -5.27098988e-07j]
 [ -1.83409258e+10 +1.27486712e-06j  -1.10135743e+10 +2.05227146e-06j]]
[[  2.73776995e+10 -5.33469210e-06j   3.

KeyboardInterrupt: 

In [ ]:
resGrid = np.asarray(np.meshgrid(np.linspace(9, 12, 50), np.linspace(0.5, 1.0, 50), np.linspace(np.pi, np.pi, 1)))
resGridLin = np.reshape(resGrid, (3,50*50)).T
dataLin = griddata(GapMapper.vertices, GapMapper.funvals, GapMapper.inverse_rescale_parameters_multiple(resGridLin))
#print(GapMapper.vertices.shape)
plt.pcolor(np.log10(np.reshape(dataLin[:,1]-dataLin[:,0], (50,50))))
